In [1]:
import os 
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from pathlib import Path
import os
import importlib
import warnings
import gc

pd.set_option('mode.chained_assignment',  None) 
warnings.simplefilter(action='ignore', category=FutureWarning) 

module_path='preprocessing'
if module_path not in sys.path:
    sys.path.append(module_path)
    
import generation
from generation import *

local = "/Users/DAHS/Desktop/early_prediction_of_circ_scl"+"/eicu-crd/preprocessing_data"
root_dir = "/Users/DAHS/Desktop/early_prediction_of_circ_scl"

In [2]:
# data = generate_adm()
# data.to_csv('check_point_data.csv.gz', compression = 'gzip')

# proc = generate_proc(data)
# proc.to_csv('check_point_proc.csv.gz', compression = 'gzip')

# out = generate_out(data)
# out.to_csv('check_point_out.csv.gz', compression = 'gzip')

data = pd.read_csv('check_point_data.csv.gz', compression = 'gzip')
proc = pd.read_csv('check_point_proc.csv.gz', compression = 'gzip')
out = pd.read_csv('check_point_out.csv.gz', compression = 'gzip')
chart = pd.read_csv('check_point_chart.csv.gz', compression = 'gzip')
labs = pd.read_csv('check_point_labs.csv.gz', compression = 'gzip')
total_proc = pd.read_csv('check_point_totalproc.csv.gz', compression = 'gzip')
meds = pd.read_csv('check_point_meds.csv.gz', compression = 'gzip')
ing = pd.read_csv('check_point_ing.csv.gz', compression = 'gzip')
vent = pd.read_csv('check_point_vent.csv.gz', compression = 'gzip')

# chart = generate_chart(data)
# chart.to_csv('check_point_chart.csv.gz', compression = 'gzip')

# labs = generate_labs(data)
# labs.to_csv('check_point_labs.csv.gz', compression = 'gzip')

# total_proc = generate_total_procedure(proc, data)
# total_proc.to_csv('check_point_totalproc.csv.gz', compression = 'gzip')

# meds = generate_meds(data)
# meds.to_csv('check_point_meds.csv.gz', compression = 'gzip')

# ing = generate_ing(data)
# ing.to_csv('check_point_ing.csv.gz', compression = 'gzip')

# vent = generate_vent(data)
# vent.to_csv('check_point_vent.csv.gz', compression = 'gzip')

In [3]:
visit = pd.read_csv("/Users/DAHS/Desktop/early_prediction_of_circ_scl"+"/eicu-crd/2.0" + "/patient.csv.gz", compression='gzip')

In [4]:
hids=visit['patientunitstayid'].unique()
print('num of patient: ', len(visit.uniquepid.unique()))
print('num of stay: ', len(hids))

num of patient:  139367
num of stay:  200859


In [5]:
hids=data['patientunitstayid'].unique()
print('num of patient: ', len(data.uniquepid.unique()))
print('num of stay: ', len(hids))

num of patient:  138479
num of stay:  196301


In [3]:
include_start_time = 24

def cohort_restirction(include_start_time, data, meds, ing, total_proc, out, chart, labs, vent):
    print("include start time",include_start_time)
    
    data=data[(data['los(hour)'] >= include_start_time)]
    hids=data['patientunitstayid'].unique()
    print('num of patient: ', len(data.uniquepid.unique()))
    print('num of stay: ', len(hids))
    
    ###MEDS

    meds=meds[meds['patientunitstayid'].isin(data['patientunitstayid'])]
    meds=meds[meds['start_time'] >= 0]
    
    ###ING

    ing=ing[ing['patientunitstayid'].isin(data['patientunitstayid'])]
    ing=ing[ing['start_time'] >= 0]

    ###PROCS

    total_proc=total_proc[total_proc['patientunitstayid'].isin(data['patientunitstayid'])]
    total_proc=total_proc[total_proc['start_time']>= 0]
        
    ###OUT

    out=out[out['patientunitstayid'].isin(data['patientunitstayid'])]
    out=out[out['start_time'] >= 0]
        
    ###CHART

    chart=chart[chart['patientunitstayid'].isin(data['patientunitstayid'])]
    chart=chart[chart['start_time'] >= 0]
        
    ###LAB

    labs=labs[labs['patientunitstayid'].isin(data['patientunitstayid'])]
    labs=labs[labs['start_time'] >= 0]
    
    ###VENT

    vent=vent[vent['patientunitstayid'].isin(data['patientunitstayid'])]
    vent=vent[vent['start_time'] >= 0]
    
    gc.collect()
    
    return  data, meds, ing, total_proc, out, chart, labs, vent

data_new, meds_new, ing_new, total_proc_new, out_new, chart_new, labs_new, vent_new = cohort_restirction(include_start_time, data, meds, ing, total_proc, out, chart, labs, vent)

include start time 24
num of patient:  105706
num of stay:  134580


In [4]:
final_meds=pd.DataFrame()
final_ing=pd.DataFrame()
final_proc=pd.DataFrame()
final_out=pd.DataFrame()
final_chart=pd.DataFrame()
final_labs=pd.DataFrame()
final_vent=pd.DataFrame()


meds_new=meds_new.sort_values(by=['start_time'])
ing_new=ing_new.sort_values(by=['start_time'])
proc_new=total_proc_new.sort_values(by=['start_time'])
out_new=out_new.sort_values(by=['start_time'])
chart_new=chart_new.sort_values(by=['start_time'])
labs_new=labs_new.sort_values(by=['start_time'])
vent_new=vent_new.sort_values(by=['start_time'])


hids=data_new['patientunitstayid'].unique()

sample_data = pd.concat([chart_new[['patientunitstayid', 'nursingchartcelltypevallabel']].rename(columns={"nursingchartcelltypevallabel":"label"}), 
                         labs_new[['patientunitstayid', 'labname']].rename(columns={"labname":"label"})], axis = 0)

In [5]:
# Specify the item_ids we are interested in
required_item_ids = {'Lactate', 'ABPs', 'ABPd'}

# Find the stay_ids that have all the required item_ids at least once
valid_stay_ids = sample_data[sample_data['label'].isin(required_item_ids)].groupby('patientunitstayid')['label'].nunique()
valid_stay_ids = valid_stay_ids[valid_stay_ids == len(required_item_ids)].index

In [6]:
meds_1 = meds_new[meds_new['patientunitstayid'].isin(valid_stay_ids)]
ing_1 = ing_new[ing_new['patientunitstayid'].isin(valid_stay_ids)]
proc_1 = proc_new[proc_new['patientunitstayid'].isin(valid_stay_ids)]
out_1 = out_new[out_new['patientunitstayid'].isin(valid_stay_ids)]
chart_1 = chart_new[chart_new['patientunitstayid'].isin(valid_stay_ids)]
labs_1 = labs_new[labs_new['patientunitstayid'].isin(valid_stay_ids)]
vent_1 = vent_new[vent_new['patientunitstayid'].isin(valid_stay_ids)]
data_1 = data_new[data_new['patientunitstayid'].isin(valid_stay_ids)]

In [7]:
hids=data_1['patientunitstayid'].unique()
print('num of patient: ', len(data_1.uniquepid.unique()))
print('num of stay: ', len(hids))

num of patient:  14760
num of stay:  15813


In [16]:
data_1['los(hour)'].sum()

2543197.0

In [17]:
import gc
gc.collect()

0

In [8]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment',  None) 
warnings.filterwarnings(action='ignore', category=pd.errors.PerformanceWarning)

import pdb

final_meds = meds_1[(meds_1['start_time']>0)&(meds_1['stop_time']>0)].copy().fillna(0)
final_ing= ing_1[(ing_1['start_time']>0)&(ing_1['stop_time']>0)].copy().fillna(0)
final_proc = proc_1.copy() 
final_out= out_1.copy()
final_chart= chart_1.copy()
final_labs= labs_1.copy()
final_vent = vent_1.copy()

feat_med = True
feat_ing = True
feat_proc = True
feat_out = True
feat_chart = True
feat_lab = True
feat_vent = True

tabularization(feat_med, feat_ing, feat_out, feat_chart, feat_lab, feat_vent,feat_proc,
                   final_meds, final_ing, final_proc, final_out, final_chart, final_labs, final_vent,
                   valid_stay_ids, data_1)

data_1.to_csv(local+'/demo.csv')

# Unique gender:  3
# Unique ethnicity:  7
Number of patient:  13616
Number of stay:  14474
Expected value of observation:  2543197.0



Tabularize EHR for total stay 16,120:   1%|▏         | 191/14474 [12:53<16:06:26,  4.06s/it]